In [1]:
import numpy as np
from dataclasses import dataclass
import math
from typing import Literal, List
import gsw  # TEOS-10 library (for salinity)
from datetime import datetime, timezone

In [2]:
from constants import *
from calibration_coefficients import *
from conversion_functions import *
from hexstrings import *

In [3]:
hexstrings = load_hexstrings_simple("basic_emb.hex")
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]

In [4]:
def temperature(byte_array, index1, index2, index3, coefs):
    F0 = 1000
    temperature_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        temperature_counts.append(count)
    temperature_counts = np.array(temperature_counts)
    temperature_counts = F0 / temperature_counts
    
    temperatures = convert_temperature(temperature_counts, coefs=coefs)
    return temperatures

In [5]:
def pressure(byte_array, index1, index2, index3, start_idx, end_idx, coefs):
    pressure_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        pressure_counts.append(count)
    pressure_counts = np.array(pressure_counts)

    temperature_counts = []
    for hex_string in hexstrings:
        temperature_value = int(hex_string[start_idx:end_idx],16)
        temperature_counts.append(temperature_value)
    temperature_counts = np.array(temperature_counts)
    
    pressure = pressure_from_frequency(pressure_counts, temperature_counts, coefs=coefs)
    return pressure

In [6]:
def conductivity(byte_array,temperatures, pressure, index1, index2, index3, coefs):
    conductivity_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        conductivity_counts.append(count)
    conductivity_counts = np.array(conductivity_counts)

    conductifitys = convert_conductivity(conductivity_counts, temperatures, pressure, coefs=coefs)
    return conductifitys

In [7]:
def salinity(conductifitys, temperatures, pressure):
    salinitys = gsw.SP_from_C(conductifitys, temperatures, pressure)
    return salinitys

In [8]:
def oxygen(temperatures, pressure, salinitys, start_idx, end_idx, coefs, units = "ml/l"):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)

    oxygens = convert_sbe43_oxygen(voltages, temperatures, pressure, salinitys1, coefs=coefs)
    
    if units == "umol/kg":
        potential_densitys = potential_density_from_t_s_p(temperatures,salinitys,pressure)
        oxygens = convert_oxygen_to_umol_per_kg(oxygens, potential_densitys) *1000
    
    return oxygens

In [9]:
def par(start_idx, end_idx, coefs):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)
    
    par = convert_par(voltages, coefs = coefs)
    return par

In [10]:
# name 1 = t090C: Temperature [ITS-90, deg C]
# name 2 = t190C: Temperature, 2 [ITS-90, deg C]

temperatures1 = temperature(bytes_list_array, 0, 1, 2, temp1_coefs)
temperatures2 = temperature(bytes_list_array, 9, 10, 11, temp2_coefs)
print("Temperatures1:", temperatures1)
print("Temperatures2:", temperatures2)

# name 0 = prDM: Pressure, Digiquartz [db]  
pressure = pressure(bytes_list_array, 6, 7, 8, 74, 77, pres_coefs)
print("Druck in psi", pressure, "gewünscht: 1.667")
pressure *= PSI_TO_DBAR # Druck von psia zu db laut seabird

print("Druck in dbar:", pressure, "gewünscht: 1.149")

# name 3 = c0mS/cm: Conductivity [mS/cm]
# name 4 = c1mS/cm: Conductivity, 2 [mS/cm]
conductifitys1 = conductivity(bytes_list_array, temperatures1, pressure, 3, 4, 5, cond1_coefs)
conductifitys2 = conductivity(bytes_list_array, temperatures2, pressure, 12, 13, 14, cond2_coefs)
print("Conductifitys1:", conductifitys1)
print("Conductifitys2:", conductifitys2)

# name 15 = sal00: Salinity, Practical [PSU]
# name 16 = sal11: Salinity, Practical, 2 [PSU]
salinitys1 = salinity(conductifitys1, temperatures1, pressure)
salinitys2 = salinity(conductifitys2, temperatures2, pressure)

print("Salinity1:", salinitys1,"PSU")
print("Salinity2:", salinitys2,"PSU")

# name 5 = sbeox0ML/L: Oxygen, SBE 43 [ml/l]
# name 6 = sbeox1ML/L: Oxygen, SBE 43, 2 [ml/l]
# name 7 = sbox0Mm/Kg: Oxygen, SBE 43 [umol/kg]
# name 8 = sbox1Mm/Kg: Oxygen, SBE 43, 2 [umol/kg]
oxygens1 = oxygen(temperatures1, pressure, salinitys1, 30, 33, oxy1_coefs, "ml/l")
oxygens2 = oxygen(temperatures2, pressure, salinitys2, 33, 36, oxy2_coefs, "ml/l")
oxygens3 = oxygen(temperatures1, pressure, salinitys1, 30, 33, oxy1_coefs, "umol/kg")
oxygens4 = oxygen(temperatures2, pressure, salinitys2, 33, 36, oxy2_coefs, "umol/kg")
print("Oxygens1:", oxygens1,"ml/l")
print("Oxygens2:", oxygens2,"ml/l")
print("Oxygens1:", oxygens3,"umol/kg")
print("Oxygens2:", oxygens4,"umol/kg")

# name 11 = par: PAR/Irradiance, Biospherical/Licor
par = par(36, 39, par_coefs)
print("Par:", par, "µmol photons m−2s−1")

Temperatures1: [3.56156397 3.56140342 3.56172452 ... 3.74795321 3.74779314 3.74800656]
Temperatures2: [3.56104427 3.56104427 3.56093531 ... 3.7434417  3.74327874 3.7434417 ]
Druck in psi [ 1.66650044  1.73668555  1.73668555 ... 34.06328618 34.12490155
 34.12490155] gewünscht: 1.667
Druck in dbar: [ 1.14901189  1.19740283  1.19740283 ... 23.4858149  23.52829721
 23.52829721] gewünscht: 1.149
Conductifitys1: [15.3307893  15.33055485 15.33078936 ... 17.6564699  17.66039193
 17.66259815]
Conductifitys2: [15.32989468 15.33007497 15.33002992 ... 17.35461101 17.351426
 17.34594621]
Salinity1: [15.55212221 15.55192778 15.55203214 ... 18.02087632 18.02530687
 18.02763887] PSU
Salinity2: [15.5513864  15.55157157 15.55157415 ... 17.68843558 17.68498031
 17.67881762] PSU
Oxygens1: [8.12350183 8.12840859 8.12832994 ... 7.871811   7.87637172 7.87620114] ml/l
Oxygens2: [8.12982213 8.129884   8.12990352 ... 7.86881094 7.86865508 7.8684949 ] ml/l
Oxygens1: [358.36352017 358.5800309  358.57653718 ... 34

In [11]:
#  CTD status
# 8-bit number from CTD
CTD_status_liste = []
bit_liste = []

for hex_string in hexstrings:
    CTD_status = format(int(hex_string[77:78], 16), '04b')
    bit = format(int(hex_string[78:80], 16), '08b')
    CTD_status_liste.append(CTD_status)
    bit_liste.append(bit)

CTD_status_liste = np.array(CTD_status_liste)
bit_liste = np.array(bit_liste)

print("CTD status:", CTD_status_liste)
print("8-Bit Zahl:", bit_liste)


CTD status: ['0011' '0011' '0011' ... '0011' '0011' '0011']
8-Bit Zahl: ['10001110' '10001111' '10010000' ... '10001001' '10001010' '10001011']


In [12]:
# name 13 = timeS: Time, Elapsed [seconds]

time_elapsed = np.array(Time_elapsed(hexstrings))
print("Time_Elapsed:",time_elapsed,"s")

Time_Elapsed: [0.00000000e+00 4.16666667e-02 8.33333333e-02 ... 1.27791667e+02
 1.27833333e+02 1.27875000e+02] s


In [13]:
# name 11 = par: PAR/Irradiance, Biospherical/Licor
voltages2 = []

for hex_string in hexstrings:
    v2 = hex_to_voltage(hex_string[36:39])
    voltages2.append(v2)
    
voltages2 = np.array(voltages2)

par = convert_par(voltages2, coefs = par_coefs)
print("Par:", par, "µmol photons m−2s−1")

Par: [9.24020531 9.26647434 9.26647434 ... 0.04538664 0.04538664 0.04538664] µmol photons m−2s−1


In [14]:
# Altimeter
voltages3 = []

for hex_string in hexstrings:
    v3 = hex_to_voltage(hex_string[39:42])
    voltages3.append(v3)
    
voltages3 = np.array(voltages3)

alt_m = 15.0 * voltages3 + 0.0
print("Voltages:", voltages3)
print("Alimeter:",alt_m)

Voltages: [1.16971917 1.17094017 1.17094017 ... 4.92185592 4.92185592 4.92063492]
Alimeter: [17.54578755 17.56410256 17.56410256 ... 73.82783883 73.82783883
 73.80952381]


In [15]:
# name 14 = dz/dtM: Descent Rate [m/s] 
pressure_test_array = np.array([1.149,1.197,1.197,1.149,1.253,1.197,1.197,1.149,1.197,1.246])
latitude = 54.50402
depths = depth_from_pressure(pressure, latitude)
print("Deaths: ",depths)



dz = np.diff(depths)
dtM = np.diff(time_elapsed)

descent_rate = dz/dtM


print("Sinkraten (m/s):", descent_rate)


Deaths:  [ 1.13870209  1.18665869  1.18665869 ... 23.27382029 23.31591673
 23.31591673]
Sinkraten (m/s): [ 1.15095843  0.         -1.15095843 ... -2.16165561  1.0103146
  0.        ]


In [16]:
navi_hex = []
latitudes = []
longitudes = []
position = []
for hex_string in hexstrings:
    navi = hex_string[60:74]
    navi_hex.append(navi)
    
navi_hex = np.array(navi_hex)

for hex_str in navi_hex:
        # Bytes extrahieren
        bytes_list = [int(hex_str[i:i+2], 16) for i in range(0, len(hex_str)-2, 2)]
        # Bytes aufsplitten
        b1, b2, b3, b4, b5, b6 = bytes_list[:6]
        last_byte_hex = hex_str[-2:]
        b7 = format(int(last_byte_hex, 16), '08b') 
        # Latitude & Longitude berechnen
        lat = ((b1 * 65536 + b2 * 256 + b3) / 50000) * (-1)**int(b7[6])
        lon = ((b4 * 65536 + b5 * 256 + b6) / 50000) * (-1)**int(b7[7])

        if b7[1] == 1:
            pos = "new"
        else: 
            pos = "old"
        position.append(pos)
        latitudes.append(lat)
        longitudes.append(lon)
position = np.array(position)
latitudes = np.array(latitudes)
longitudes = np.array(longitudes)
print("Position:", position)  
print("Latitudes:", latitudes)
print("Longitudes:",longitudes)

Position: ['old' 'old' 'old' ... 'old' 'old' 'old']
Latitudes: [54.15508 54.15508 54.15508 ... 54.15502 54.15502 54.15502]
Longitudes: [11.29326 11.29326 11.29326 ... 11.29332 11.29332 11.29332]


In [17]:
# name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]
voltages4 = []

for hex_string in hexstrings:
    v4 = hex_to_voltage(hex_string[42:45])
    voltages4.append(v4)
    
voltages4 = np.array(voltages4)

flurometer = convert_eco(voltages4, coefs = fluroeco_coefs)
print("Fluorescence:" , flurometer, "mg/m^3")

Fluorescence: [1.35889377 1.35889377 1.33691575 ... 1.77647619 1.75449817 1.75449817] mg/m^3


In [18]:
# name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]
voltages5 = []

for hex_string in hexstrings:
    v5 = hex_to_voltage(hex_string[45:48])
    voltages5.append(v5)
    
voltages5 = np.array(voltages5)

turbidity = convert_eco(voltages5, coefs = turbidity_coefs)
print("Turbidity:", turbidity, "NTU")

Turbidity: [0.52689866 0.52689866 0.53178266 ... 2.95669109 2.9664591  2.96157509] NTU


In [19]:
# name 12 = spar: SPAR, Biospherical/Licor

voltages9 = []

for hex_string in hexstrings:
    N = int(hex_string[57:60],16)
    v9 = N / 819
    voltages9.append(v9)
    
voltages9 = np.array(voltages9)

spar = convert_spar(voltages9, coefs = spar_coefs)
print("SPar:", spar, "µmol photons m−2s−1")

SPar: [366.85335775 366.85335775 366.85335775 ... 388.43296703 386.47118437
 386.47118437] µmol photons m−2s−1


In [20]:
# ???
fluroeco_coefs2 = ECOCoefficients()
fluroeco_coefs2.slope = 1.000
fluroeco_coefs2.offset = 0.000


voltages7 = []

for hex_string in hexstrings:
    v7 = hex_to_voltage(hex_string[51:54])
    voltages7.append(v7)
    
voltages7 = np.array(voltages7)

flurometer = convert_eco(voltages7, coefs = fluroeco_coefs2)
print("Fluorescence:" , flurometer, "mg/m^3")
print("Why???????")

# Altimeter
voltages3 = []

for hex_string in hexstrings:
    v3 = hex_to_voltage(hex_string[39:42])
    voltages3.append(v3)
    
voltages3 = np.array(voltages3)

alt_m = 15.0 * voltages3 + 0.0

print(alt_m)

Fluorescence: [0.07936508 0.08058608 0.07936508 ... 0.07936508 0.07936508 0.07814408] mg/m^3
Why???????
[17.54578755 17.56410256 17.56410256 ... 73.82783883 73.82783883
 73.80952381]


In [21]:
# name 17 = flag:  0.000e+00

flag = np.zeros_like(hexstrings, dtype=float)
print(flag)

[0. 0. 0. ... 0. 0. 0.]


In [22]:
# Date and Time
swapped = [
    "".join([s[-8:][i:i+2] for i in range(0, 8, 2)][::-1])
    for s in hexstrings
]
timestamps = [int(x, 16) for x in swapped]    
datetimes = [datetime.fromtimestamp(ts, tz=timezone.utc) for ts in timestamps]
datetimes_formatted = np.array([d.strftime("%Y-%m-%d %H:%M:%S") for d in datetimes])
print("Date and Times", datetimes_formatted)

Date and Times ['2024-02-08 08:31:24' '2024-02-08 08:31:24' '2024-02-08 08:31:25' ...
 '2024-02-08 08:33:32' '2024-02-08 08:33:32' '2024-02-08 08:33:32']


In [23]:
print("Pressures:", pressure,"db\n")

print("Temperatures1:", temperatures1,"°C\n")
print("Temperatures2:", temperatures2,"°C\n")

print("Conductifitys1:", conductifitys1,"mS/cm\n")
print("Conductifitys2:", conductifitys2,"mS/cm\n")

print("Oxygens1:", oxygens1,"ml/l\n")
print("Oxygens2:", oxygens2,"ml/l\n")
print("Oxygens3:", oxygens3,"umol/kg\n")
print("Oxygens4:", oxygens4,"umol/kg\n")

print("Fluorescence:" , flurometer, "mg/m^3\n")

print("Turbidity:", turbidity, "NTU\n")

print("Par:", par, "µmol photons m−2s−1\n")
print("SPar:", spar, "µmol photons m−2s−1\n")

print("Time_Elapsed:", time_elapsed,"s\n")

print("Salinity1:", salinitys1,"PSU\n")
print("Salinity2:", salinitys2,"PSU\n")
print("Flag:", flag,"\n")

Pressures: [ 1.14901189  1.19740283  1.19740283 ... 23.4858149  23.52829721
 23.52829721] db

Temperatures1: [3.56156397 3.56140342 3.56172452 ... 3.74795321 3.74779314 3.74800656] °C

Temperatures2: [3.56104427 3.56104427 3.56093531 ... 3.7434417  3.74327874 3.7434417 ] °C

Conductifitys1: [15.3307893  15.33055485 15.33078936 ... 17.6564699  17.66039193
 17.66259815] mS/cm

Conductifitys2: [15.32989468 15.33007497 15.33002992 ... 17.35461101 17.351426
 17.34594621] mS/cm

Oxygens1: [8.12350183 8.12840859 8.12832994 ... 7.871811   7.87637172 7.87620114] ml/l

Oxygens2: [8.12982213 8.129884   8.12990352 ... 7.86881094 7.86865508 7.8684949 ] ml/l

Oxygens3: [358.36352017 358.5800309  358.57653718 ... 346.59229751 346.7918986
 346.78375911] umol/kg

Oxygens4: [358.64253525 358.64521275 358.64607122 ... 346.5502889  346.54435893
 346.5389793 ] umol/kg

Fluorescence: [0.07936508 0.08058608 0.07936508 ... 0.07936508 0.07936508 0.07814408] mg/m^3

Turbidity: [0.52689866 0.52689866 0.53178266 

In [24]:
name 0 = Pressure: -42139.332086015915 db                        --> Falsch
name 1 = Temp_1: 3.561563969710619 °C                     --> Richtig
name 2 = Temp_2: 3.561044266345732 °C                     --> Richtig
name 3 = Conductivity_1: 15.330789300015969 mS/cm         --> Richtig
name 4 = Conductivity_2: 15.32989468343113 mS/cm          --> Richtig
name 5 = Oxygen1: [8.12350182] ml/l                       --> Richtig
name 6 = Oxygen2: [8.12986258] ml/l                       --> Richtig
name 7 = Oxygen1: [358.3635196] umol/kg                   --> Richtig
name 8 = Oxygen2: [358.64431987] umol/kg                  --> Richtig
name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]   --> Richtig
name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]             --> Richtig
name 11 = par: PAR/Irradiance, Biospherical/Licor         --> Richtig
name 12 = spar: SPAR, Biospherical/Licor                         --> Richtig
name 13 = timeS: Time, Elapsed [seconds]                  --> Richtig

name 14 = dz/dtM: Descent Rate [m/s]                            --> Falsch

name 15 = Salinity1: 15.5521222127295 PSU                 --> Richtig
name 16 = Salinity2: 15.551386401977785 PSU               --> Richtig
name 17 = flag:  0.000e+00                                --> Richtig


name 0 = Pressure: -42139.332086015915 db                        --> Falsch


name 14 = dz/dtM: Descent Rate [m/s]                             --> hängt stark von pressure ab, zu ungenau durch runden bei test_array

SyntaxError: invalid character '°' (U+00B0) (950467276.py, line 2)